# DEMO for Whole Robot Motion

In [ ]:
import sys
import os

import pandas as pd
import numpy as np
import json

from pydrake.geometry import ( StartMeshcat )
from pydrake.geometry.optimization import (LoadIrisRegionsYamlFile) # type: ignore

# add parent folders to path
sys.path.append(os.path.abspath('..'))

# Update the import
from src.auxiliar_functions import (VisualizeConnectivity)
from src.gcs_traj_opt_arduino import (demo_trajectory_optimization, demo_trajectory_between_positions, record_np_trajectory)
from src.gcs_helper import VisualizeRegions

# Start the visualizer (only run this once).
meshcat = StartMeshcat()

Loading seeds and IRIS regions.

In [ ]:
parent_dir = os.path.abspath('..') # path to the parent directory
iris_regions = dict()

gcs_seeds_path = os.path.join(parent_dir, 'common-files','demo_seeds.json')
regions_path = os.path.join(parent_dir, 'common-files/regions/my_regions2.yaml')

if os.path.isfile(regions_path):
    iris_regions.update(LoadIrisRegionsYamlFile(regions_path))
    print(f"Loaded iris regions from {regions_path}.")
else:
    print(f"{regions_path} not found. No previously computed regions were loaded.")


with open(gcs_seeds_path, 'r') as f: # load the seeds
    iris_seeds = json.load(f)    

Creating the connection with ARDUINO

In [ ]:
from src.data_streamer import DataStreamer
try:
    streamer = DataStreamer('/dev/ttyACM0', True)
    streamer.open_connection()
except Exception as e:
    print(f"Error opening connection: {e}")  

Run GCS for random seeds.

In [ ]:
history, indexes_list = demo_trajectory_optimization(iris_regions, iris_seeds,meshcat, streamer)

Saving joint history and indexes

In [ ]:
# Save `history` to a CSV file
np.savetxt("history.csv", history, delimiter=",")

# Save `indexes_list` to a CSV file|
pd.DataFrame(indexes_list, columns=["Indexes"]).to_csv("indexes_list.csv", index=False)

dataframe = pd.read_csv("history.csv", header=None)
dataframe.head()

In [ ]:
indexes_list

Graph

In [ ]:
VisualizeConnectivity(iris_regions)

Visualizing collision-free regions.

In [ ]:
VisualizeRegions(meshcat, iris_regions)

Code for running GCS for two desired positions

In [ ]:
import numpy as np
q_start = np.array([0.0, 0.0, 0.0, 0.0, 0.0])
q_goal = np.array([1.57, -0.12, -0.89, -0.76, 0])

demo_trajectory_between_positions(q_start, q_goal, iris_regions, meshcat,streamer)

In [ ]:
streamer.close_connection()

In [ ]:
list_of_positions = [[0.0, 0.7, -1.3, -0.9, 0.0],
[0.64, 0.53, -1.30, -1.15,0.0],
[1.40, 0.53, -1.30, -1.15,0.0],
[1.5708, -0.12, -0.89, -0.76, 0],
[0.71, 0.91, -1.25, -1.45, 0.0],
[-1.5708, 0.91, -1.25, -1.45, 0.0],
[-1.57, -0.12, -0.89, -0.76, 0],
[-1.57, 0.65, -1.0, -1.57, 0.0],
[-0.25, 0.0, 0.0, 0.0, 0.0]]

second_list_of_positions = [
	[0.0, 0.0, 0.0, 0.0, 0.0],
	[0.0, 0.0, 0.0, 0.0, 0.0],
	[0.2, 0.7, -1.3, -0.9, 0.0],
	[0.2, 0.95, -1.33, -1.27, 0.0],
	[1.57, 1.57, -1.57, -1.57, 0.0],
	[0.2, 0.95, -1.33, -1.27, 0.0],
	[0.2, 0.7, -1.3, -0.9, 0.0],
	[0.2, 0.95, -1.33, -1.27, 0.0],
	[1.57, 1.57, -1.57, -1.57, 0.0],
	[0.1, 0.95, -1.33, -1.27, 0.0],
	[0.1 , 0.7, -1.3, -0.9, 0.0],
	[0.1, 0.95, -1.33, -1.27, 0.0],
	[1.57, 1.57, -1.57, -1.57, 0.0],
	[0.1, 0.95, -1.33, -1.27, 0.0],
	[0.1, 0.7, -1.3, -0.9, 0.0],
	[0.1, 0.95, -1.33, -1.27, 0.0],
	[1.57, -0.12, -0.89, -0.76, 0.0],
	[0.1, 0.0, 0.0, 0.0, 0.0]
]

record_np_trajectory(second_list_of_positions, meshcat)

In [ ]:
import pandas as pd

# Load the CSV file
history = pd.read_csv("/home/arath/github/arms-robotic-system/outputs/history4.csv", header=None)
indexes_list = pd.read_csv("/home/arath/github/arms-robotic-system/outputs/indexes_list4.csv")

history, indexes_list

# Convert the DataFrames to NumPy arrays
history_np = history.to_numpy()
indexes_list_np = indexes_list.to_numpy()
# Print the shapes of the arrays
print("Shape of history array:", history_np.shape)
print("Shape of indexes_list array:", indexes_list_np.shape)



for i in range(len(indexes_list_np)-1):
    print(history_np[:, indexes_list_np[i]].flatten())

print(history_np[:, indexes_list_np[-1]-1].flatten())